In [28]:
# fast screen replacement with colour + texture harmonisation
# and *minimal* edge-antialias blur (no fade)
# -----------------------------------------------------------
import cv2
import json
import numpy as np
import logging
import math
from skimage.exposure import match_histograms

TRACK_JSON = "aruco_screen_tracks.json"
VIDEO_IN = "aruco.mp4"
MEDIA_IMG = "simono.jpg"
VIDEO_OUT = "screen_with_media.mp4"

# ─ look-&-feel dials ──────────────────────────────────────
ALPHA = 0.0  # global fade 0-1
CT_STRENGTH = 0.42  # chroma/contrast transfer
HM_STRENGTH = 0.13  # luminance histogram blend
FEATHER_PX = 2  # σ of tiny Gaussian edge blur (anti-alias)
MAX_BLUR_PX = 3.0  # cap for soft-blur
NOISE_GAIN = 1.0  # 1 = match screen grain

# ─ logging (optional) ────────────────────────────────────
logging.basicConfig(
    filename="replace_debug.log",
    filemode="w",
    level=logging.DEBUG,
    format="%(message)s",
)
log = logging.getLogger("replace")


# ═ helpers ═══════════════════════════════════════════════
def get_mean_and_std(x):
    m, s = cv2.meanStdDev(x)
    return np.hstack(m).astype(np.float32), np.hstack(s).astype(np.float32)


def color_transfer(src, tgt, s):
    if s <= 0:
        return src
    src_lab = cv2.cvtColor(src, cv2.COLOR_BGR2LAB).astype(np.float32)
    tgt_lab = cv2.cvtColor(tgt, cv2.COLOR_BGR2LAB).astype(np.float32)
    m0, s0 = get_mean_and_std(src_lab)
    m1, s1 = get_mean_and_std(tgt_lab)
    s0[s0 == 0] = 1.0
    scaled = (src_lab - m0) * (s1 / s0) + m1
    out = cv2.addWeighted(src_lab, 1 - s, scaled, s, 0)
    return cv2.cvtColor(out.clip(0, 255).astype(np.uint8), cv2.COLOR_LAB2BGR)


def match_luminance(src, tgt, s):
    if s <= 0:
        return src
    sl = cv2.cvtColor(src, cv2.COLOR_BGR2LAB)
    tl = cv2.cvtColor(tgt, cv2.COLOR_BGR2LAB)
    matched = match_histograms(sl[..., 0], tl[..., 0], channel_axis=None)
    blend = cv2.addWeighted(
        sl[..., 0].astype(np.float32), 1 - s, matched.astype(np.float32), s, 0
    )
    sl[..., 0] = np.clip(blend, 0, 255).astype(np.uint8)
    return cv2.cvtColor(sl, cv2.COLOR_LAB2BGR)


def est_sharp(i):
    return cv2.Laplacian(i, cv2.CV_32F).var()


def harmonise_texture(patch, ref, max_blur_px, noise_gain):
    sp, sr = est_sharp(patch), est_sharp(ref)
    if sp > sr:
        sigma = min(max_blur_px, 0.7 * math.sqrt(sp / sr - 1))
        if sigma > 0.4:
            patch = cv2.GaussianBlur(patch, (0, 0), sigma)
    ref_blur = cv2.GaussianBlur(ref, (0, 0), 3)
    n_std = noise_gain * (ref.astype(np.int16) - ref_blur.astype(np.int16)).std()
    if n_std > 0.5:
        noise = np.random.normal(0, n_std, patch.shape).astype(np.float32)
        patch = np.clip(patch.astype(np.float32) + noise, 0, 255).astype(np.uint8)
    return patch


def sample_screen(frame, dst, size):
    pw, ph = size
    src_q = np.array([[0, 0], [pw - 1, 0], [pw - 1, ph - 1], [0, ph - 1]], np.float32)
    H = cv2.getPerspectiveTransform(dst, src_q)
    return cv2.warpPerspective(frame, H, (pw, ph), flags=cv2.INTER_LINEAR)


def build_patch_mask(h, w, sigma):
    """255 rectangle blurred by tiny σ for anti-alias."""
    mask = np.full((h, w), 255, np.uint8)
    if sigma > 0:
        mask = cv2.GaussianBlur(mask, (0, 0), sigma)
    return mask


# ═ I/O ═══════════════════════════════════════════════════
with open(TRACK_JSON) as f:
    meta = json.load(f)
frames = meta["frames"]
FW, FH = meta["frame_width"], meta["frame_height"]

cap = cv2.VideoCapture(VIDEO_IN)
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(VIDEO_OUT, cv2.VideoWriter_fourcc(*"mp4v"), fps, (FW, FH))

media = cv2.imread(MEDIA_IMG)
mh, mw = media.shape[:2]
src_quad = np.zeros((4, 2), np.float32)

# ═ main loop ═════════════════════════════════════════════
for rec in frames:
    ok, frame = cap.read()
    if not ok:
        break
    if not rec["valid"]:
        out.write(frame)
        continue

    dst = np.array(rec["corners"], np.float32)
    ov_w = int(np.linalg.norm(dst[1] - dst[0]) + 0.5)
    ov_h = int(np.linalg.norm(dst[3] - dst[0]) + 0.5)

    # overscale + crop
    s = max(ov_w / mw, ov_h / mh) * 1.01
    big = cv2.resize(media, (math.ceil(mw * s), math.ceil(mh * s)), cv2.INTER_AREA)
    x0 = (big.shape[1] - ov_w) // 2
    y0 = (big.shape[0] - ov_h) // 2
    patch = big[y0 : y0 + ov_h, x0 : x0 + ov_w]

    scr = sample_screen(frame, dst, (ov_w, ov_h))
    patch = color_transfer(patch, scr, CT_STRENGTH)
    patch = match_luminance(patch, scr, HM_STRENGTH)
    patch = harmonise_texture(patch, scr, MAX_BLUR_PX, NOISE_GAIN)

    soft = build_patch_mask(ov_h, ov_w, FEATHER_PX)

    ph, pw = patch.shape[:2]
    src_quad[:] = [[0, 0], [pw - 1, 0], [pw - 1, ph - 1], [0, ph - 1]]
    H = cv2.getPerspectiveTransform(src_quad, dst)
    warped = cv2.warpPerspective(patch, H, (FW, FH))
    warped_mask = cv2.warpPerspective(soft, H, (FW, FH))

    alpha = (warped_mask.astype(np.float32) / 255.0)[..., None]
    blended = (
        warped.astype(np.float32) * alpha + frame.astype(np.float32) * (1 - alpha)
    ).astype(np.uint8)
    if ALPHA:
        blended = cv2.addWeighted(frame, 1 - ALPHA, blended, ALPHA, 0)
    out.write(blended)

cap.release()
out.release()
print("✅  Media burned-in with tiny edge-blur — saved to", VIDEO_OUT)


✅  Media burned-in with tiny edge-blur — saved to screen_with_media.mp4
